# Summarization of a Resume <br>
** Summarization is an imporatnt task undeer text analytics. It captures all the relevent and significant information from the data hence reducing the effort to go through the data. <br>
In the context of resume clssification, resume summarization would help in reducing the size of the data to what actually helps in classification by eliminating all noisy textual part.<br>
At the end of summarization, it is seen that whatever noisy data these resumes had is completely eliminated (eg: advertisements, website of the source etc)**

In [195]:
import pandas as pd 
import numpy as np 
import re
import nltk
from nltk.corpus import stopwords
from wordcloud import STOPWORDS
from nltk.probability import FreqDist
import random
import math

In [196]:
df = pd.read_csv("./clean_data1.csv")
print(df.shape)
#df.head()
df = df[["ID","Category","newer_res","Resume"]]

(1199, 5)


In [197]:
df.head()

,ID,Category,newer_res,Resume
0,1,HR,"john h. smith , p.h.r . 800-991-5187 | po box ...","b'John H. Smith, P.H.R.\n800-991-5187 | PO Box..."
1,2,HR,name surname address mobile no/email personal ...,b'Name Surname\nAddress\nMobile No/Email\nPERS...
2,3,HR,anthony brown hr assistant areas of expertise ...,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3,4,HR,www.downloadmela.com satheesh email id : caree...,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4,5,HR,human resources director expert organizational...,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert..."


In [198]:
resume = df["newer_res"][math.floor(random.random()*(df.shape)[0])]
resume

'cynthia perreira 5302 trolley square crossingatlanta , ga 30306404-202-8111email : gtg504j @ mail.gatech.edu objective to obtain challenging interesting summer internship mechanical engineering education georgia institute technology atlanta , ga aug. 2002present : candidate m.s . thesis may 2004 mechanical engineering specialization design controls coursework progress : mechatronics , designing open engineering systems coursework completed : linear control systems , engineering design , numerical methods graduate research assistant : advisors dr. wayne j . book ( intelligent machine dynamics laboratory ) dr. chris paredis ( systems realization laboratory ) . currently creating systematic method determining whether hydraulic electrical power best given design problem . cumulative gpa : 3.66/4.0 massachusetts institute technology cambridge , ma sept. 1998june 2002 : graduated b.s . mechanical engineering minors biomedical engineering economics mechanical engineering : product eng . proc

In [199]:
df["Category"].unique()

array(['HR', 'Designing', 'Managment', 'Information Technology',
       'Education', 'Advocate', 'Business Development',
       'Health & Fitness', 'Agricultural', 'BPO', 'Sales', 'Consultant',
       'Digital Media', 'Automobile', 'Food & Beverages', 'Finance',
       'Apparel', 'Engineering', 'Accountant', 'Building & Construction',
       'Architects', 'Public Relations', 'Banking', 'Arts', 'Aviation'],
      dtype=object)

In [200]:
token = nltk.word_tokenize(resume)
use_data = nltk.pos_tag(resume)
token[:10]
print(use_data[:10])

[('c', 'NNS'), ('y', 'VBP'), ('n', 'JJ'), ('t', 'NN'), ('h', 'NN'), ('i', 'VBZ'), ('a', 'DT'), (' ', 'JJ'), ('p', 'NN'), ('e', 'NN')]


** Summarization is based on scoring every sentence based on criteria like weightage given to each word and selecting the sentances with most weightage. The sentence count set here is 15. **
<br><br>
**HOW IT WORKS**
* Frequency distributuon of workd is per document is found.
* Score for each sentence is found
* Each sentence in the document is scored based on the words present in it.
* The score given to each word is proportional to the count of its occurances.
* The higher the score, thre more important is the sentence.
* Top 15 senetences are the selected and presented.

In [201]:
from nltk.probability import FreqDist
from string import punctuation

def summarize(doc,words):
    score={}
    fd = FreqDist(words)
    for i,t in enumerate(doc):
        score[i] = 0
        for j in nltk.word_tokenize(t):
            if j in fd:
                score[i]+=fd[j]
    
    r = sorted(list(score.items()),key=lambda x:x[1],reverse=True)[:math.floor(0.60*len(doc))]
    r.sort(key=lambda x:x[0])
    l = [doc[i[0]] for i in r]
    return "\n\n".join(l)


In [202]:
sent = nltk.sent_tokenize(resume)
puncu = punctuation
word_token = nltk.word_tokenize(resume)
resume_new = " ".join([i for i in word_token if i not in puncu])

** Actual Resume **

In [203]:
resume

'cynthia perreira 5302 trolley square crossingatlanta , ga 30306404-202-8111email : gtg504j @ mail.gatech.edu objective to obtain challenging interesting summer internship mechanical engineering education georgia institute technology atlanta , ga aug. 2002present : candidate m.s . thesis may 2004 mechanical engineering specialization design controls coursework progress : mechatronics , designing open engineering systems coursework completed : linear control systems , engineering design , numerical methods graduate research assistant : advisors dr. wayne j . book ( intelligent machine dynamics laboratory ) dr. chris paredis ( systems realization laboratory ) . currently creating systematic method determining whether hydraulic electrical power best given design problem . cumulative gpa : 3.66/4.0 massachusetts institute technology cambridge , ma sept. 1998june 2002 : graduated b.s . mechanical engineering minors biomedical engineering economics mechanical engineering : product eng . proc

In [204]:
summ = summarize(sent,resume_new)

** Summarized Resume **

In [205]:
print(summ)

cynthia perreira 5302 trolley square crossingatlanta , ga 30306404-202-8111email : gtg504j @ mail.gatech.edu objective to obtain challenging interesting summer internship mechanical engineering education georgia institute technology atlanta , ga aug. 2002present : candidate m.s .

thesis may 2004 mechanical engineering specialization design controls coursework progress : mechatronics , designing open engineering systems coursework completed : linear control systems , engineering design , numerical methods graduate research assistant : advisors dr. wayne j .

book ( intelligent machine dynamics laboratory ) dr. chris paredis ( systems realization laboratory ) .

currently creating systematic method determining whether hydraulic electrical power best given design problem .

cumulative gpa : 3.66/4.0 massachusetts institute technology cambridge , ma sept. 1998june 2002 : graduated b.s .

mechanical engineering minors biomedical engineering economics mechanical engineering : product eng .


In [209]:
len(sent)

35